# The Model

## reloading
sometimes it helps to reload data -- redo imports up here

In [ ]:
from importlib import reload

In [ ]:
import neural_net as nen
reload(nen)
from neural_net import *

## setup
Import, load data and instantiate relevant classes (DataLoader, Dataset etc)

In [ ]:
## imports
#  modules
import numpy as np
import pandas as pd
import xarray as xr
import warnings
import os

import torch
from torch import nn

# custom code
from dask_addons import FlattenedDaskDataset
from neural_net import DNN, geoDataset, BatchDataLoader, train, test
from open_fire_data import FlattenedTruthTable

warnings.filterwarnings('ignore')

In [ ]:
from dask.distributed import Client
client = Client()
client

2025-08-07 21:22:14,760 - distributed.scheduler - WARNING - Detected different `run_spec` for key ('rechunk-split-0fc45d5b86da226cc122bcb4970315c5', 183) between two consecutive calls to `update_graph`. This can cause failures and deadlocks down the line. Please ensure unique key names. If you are using a standard dask collections, consider releasing all the data before resubmitting another computation. More details and help can be found at https://github.com/dask/dask/issues/9888. 
Debugging information
---------------------
old task state: released
old run_spec: Alias(('rechunk-split-0fc45d5b86da226cc122bcb4970315c5', 183)->('getitem-rechunk-split-0fc45d5b86da226cc122bcb4970315c5', 183))
new run_spec: <Task ('rechunk-split-0fc45d5b86da226cc122bcb4970315c5', 183) getitem(...)>
old dependencies: {('getitem-rechunk-split-0fc45d5b86da226cc122bcb4970315c5', 183)}
new dependencies: frozenset({('getitem-83af0cb3d308ebcd323c15a7659ef338', 1, 0, 1)})



In [ ]:
DATA_DIR_PATH = '../data/_ZARR_FILES'

data_path = os.path.join(DATA_DIR_PATH, 'alaska_full.zarr')
prior_data_path = os.path.join(DATA_DIR_PATH, 'alaska_prior.zarr')

data = xr.open_zarr(data_path, decode_timedelta=False)
prior_data = xr.open_zarr(prior_data_path, decode_timedelta=False)

In [ ]:
input_ds = FlattenedDaskDataset(data, prior_data)
input_ds.setup()

In [ ]:
fire_data = FlattenedTruthTable(
    pd.read_csv('../data/_FIRE/alaska_range_csv/data.csv'),
    lat_vals = input_ds.latitude,
    long_vals = input_ds.longitude
)

In [ ]:
ds = geoDataset(input_ds, fire_data, feature_num = 13)

## training


In [ ]:
model = DNN()
print(model)

In [ ]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
train_dataloader = BatchDataLoader(ds, end=100)
test_dataloader = BatchDataLoader(ds)

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
len(train_dataloader)

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer, device)
    test(test_dataloader, model, loss_fn, device)
print("Done!")